# Model Selection and Validation Notebook

In this notebook you will see the code of the Model Selection and Model Validation sessions.

We will use the swiss dataset for explaining this variable selection process. We can import it from a URL as follows:

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

data_url = "https://vincentarelbundock.github.io/Rdatasets/csv/datasets/swiss.csv"
swiss = pd.read_csv(data_url, index_col=0)

In [ ]:
swiss.head()

In previous sessions we have seen the Backwards stepwise selection manually. We can start with the full model and check which variable to eliminate using the p-values of the beta parameters. Let's start creating the full model:

In [ ]:
swiss["Infant_Mortality"] = swiss["Infant.Mortality"]
swiss.head()

In [ ]:
model_1 = smf.ols(formula = "Fertility ~ Agriculture + Examination + Education + Catholic + Infant_Mortality",
                  data = swiss).fit()
print(model_1.summary())

We see that the variable with the greater p-value is "Examination", so we will create a `model_2` eliminating that variable.

In [ ]:
model_2 = smf.ols(formula = "Fertility ~ Agriculture + Education + Catholic + Infant_Mortality",
                  data = swiss).fit()
print(model_2.summary())

This could be a good model for us just using the variable selection strategy that we have studied so far. In this notebook we will see several strategies for variable selection, so we can compare them and take better informed decisions.

## ANOVA method

We will compare different models using ANOVA depending on the difference on their F statistic result.

In [ ]:
model_0 = smf.ols(formula = "Fertility ~ 1", data = swiss).fit()
print(model_0.summary())

In [ ]:
from statsmodels.stats.anova import anova_lm
anova_lm(model_0, model_1)

In [ ]:
anova_lm(model_2, model_1)

## Best Subset Regression

There is not a specific function in Python for obtaining the results of a best subset regression, so we have to create a program for performing those operations (this could be a good programming exercise if you want to try by yourselves). Below, you have my solution for this best subset regression function:

In [ ]:
import itertools

# Function to fit a model and return the adjusted R-squared
def fit_model_and_calc_aic(y, X):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return {"aic" : model.aic, "bic" : model.bic, "r2" : model.rsquared_adj}

def best_subset_reg(df, y, predictors, metric = "aic"):
  # Initialize the best subset and its performance
  best_subset = []
  best_subset_metric = -np.inf
  best_subset_metric_aic = np.inf
  all_subsets = []
  all_models = []

  # Iterate over all possible subsets of predictors
  for subset_size in range(1, len(predictors) + 1):
      for subset in itertools.combinations(predictors, subset_size):
          X = df[list(subset)]
          model = fit_model_and_calc_aic(y, X)
          all_subsets.append(subset)
          all_models.append(model)
          current_metric = model[metric]

          if (metric == "r2") and (current_metric > best_subset_metric):
              best_subset = subset
              best_subset_metric = current_metric
          elif (metric != "r2") and (current_metric < best_subset_metric_aic):
              best_subset = subset
              best_subset_metric_aic = current_metric

  print("Best subset:", best_subset)
  print(metric, ":", best_subset_metric_aic)
  bestsubreg = pd.DataFrame(all_subsets, all_models)
  return(bestsubreg)


With this function we can print an specific statistic that we want to evaluate, but all the results will be stored in the return of the function, so we can visualize every step of the algorithm.

In [ ]:
# Target variable
y = swiss['Fertility']

# Define the predictor variables
predictors = ['Agriculture', 'Examination', 'Education', 'Catholic', 'Infant_Mortality']

# Use the function
allsub = best_subset_reg(df = swiss, y = y, predictors = predictors, metric = "aic")

In [ ]:
allsub

## Stepwise method

Again, there is not an algorithm in statsmodels that allow us to perform the stepwise selection. We have two options, we can do it by hand or using a different library named “mlxtend”. Both options are used in the notebook.

In [ ]:
def forward_selection(data, target, significance_level=0.05):
    initial_features = data.columns.tolist()
    best_features = []
    while len(initial_features) > 0:
        remaining_features = list(set(initial_features) - set(best_features))
        new_pval = pd.Series(index=remaining_features, dtype='float64')
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if min_p_value < significance_level:
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features

def backward_elimination(data, target, significance_level=0.05):
    features = data.columns.tolist()
    while len(features) > 0:
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if max_p_value >= significance_level:
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break
    return features

def bidirectional_stepwise(data, target, significance_level=0.05):
    forward_features = forward_selection(data, target, significance_level)
    return backward_elimination(data[forward_features], target, significance_level)


In [ ]:
# Define the target variable and predictors
target = swiss['Fertility']
predictors = swiss.drop(columns='Fertility')

# Perform forward selection
forward_selected_features = forward_selection(predictors, target)
print("Forward Selection: ", forward_selected_features)

# Perform backward elimination
backward_eliminated_features = backward_elimination(predictors, target)
print("Backward Elimination: ", backward_eliminated_features)

# Perform bidirectional stepwise regression
bidirectional_features = bidirectional_stepwise(predictors, target)
print("Bidirectional Stepwise: ", bidirectional_features)


In [ ]:
swiss["Infant_Mortality"] = swiss["Infant.Mortality"]

In [ ]:
#!pip install --upgrade mlxtend
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [ ]:
from sklearn.linear_model import LinearRegression
#from mlxtend.feature_selection import SequentialFeatureSelector as SFS

target = swiss['Fertility']
predictors = swiss.drop(columns='Fertility')

linear_regression = LinearRegression()

# Forward Selection
forward_selector = SFS(linear_regression,
                       k_features="best",
                       forward=True,
                       floating=False,
                       scoring='r2',
                       cv=0)
forward_selector.fit(predictors, target)
forward_selected_features = list(predictors.columns[list(forward_selector.k_feature_idx_)])
print("Forward Selection: ", forward_selected_features)

# Backward Elimination
backward_selector = SFS(linear_regression,
                        k_features="best",
                        forward=False,
                        floating=False,
                        scoring='r2',
                        cv=0)
backward_selector.fit(predictors, target)
backward_eliminated_features = list(predictors.columns[list(backward_selector.k_feature_idx_)])
print("Backward Elimination: ", backward_eliminated_features)


In [ ]:
def summarize_results(selector, method, predictors):
    selected_features = list(predictors.columns[list(selector.k_feature_idx_)])
    print(f"{method} Results:")
    print("Selected features:", selected_features)
    print("Number of features:", selector.k_feature_names_)
    print("R-squared:", selector.k_score_)
    print("\nFeature Selection History:")
    for idx, values in selector.subsets_.items():
        print("Step", idx, ": Features", list(predictors.columns[list(values["feature_idx"])]), "- R-squared:" ,values["avg_score"])

# Summarize Forward Selection results
summarize_results(forward_selector, "Forward Selection", predictors)
print("\n")
# Summarize Backward Elimination results
summarize_results(backward_selector, "Backward Elimination", predictors)


# Model Validation

Using the best model, now we can obtain the validated scores using the methods we have learned at the beginnig of this module: Holdout (train-test), LOOCV and K-Fold.

In [ ]:
from sklearn.model_selection import train_test_split, LeaveOneOut, KFold, cross_val_score
from sklearn.linear_model import LinearRegression

target = swiss['Fertility']
predictors = swiss.drop(columns=['Fertility', 'Examination']) # we eliminate in this case Examination

## Holdout

In [ ]:
# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=2023)

# Create a linear regression model and fit it on the training data
lr_model = LinearRegression().fit(X_train, y_train)

# Calculate the R-squared score on the test data
holdout_score = lr_model.score(X_test, y_test)
print(f"Holdout R-squared: {holdout_score}")

## LOOCV

the R-squared score is not well-defined when there is only one sample in the test set, which is the case with Leave-One-Out Cross-Validation (LOOCV). In this situation, it's better to use a different scoring metric, such as the mean squared error (MSE) or mean absolute error (MAE).

But in order to compare it, I have developed a program that manually calculates the R-squared for each iteration of the LOOCV method, giving the final average R-squared.

In [ ]:
# Create a LeaveOneOut object
loo = LeaveOneOut()

# Create a linear regression model
lr_model = LinearRegression()

# Keep track of true and predicted values
y_true = []
y_pred = []

# Iterate through each split in the LOOCV
for train_index, test_index in loo.split(predictors):
    X_train, X_test = predictors.iloc[train_index], predictors.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

    # Fit the model on the training data
    lr_model.fit(X_train, y_train)

    # Make predictions on the test data
    predictions = lr_model.predict(X_test)

    # Append the true and predicted values
    y_true.extend(y_test)
    y_pred.extend(predictions)

# Convert the true and predicted values to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute the R-squared manually
loocv_r2 = 1 - ((y_true - y_pred) ** 2).sum() / ((y_true - y_true.mean()) ** 2).sum()
print(f"LOOCV R-squared: {loocv_r2}")


## K-Fold

In [ ]:
# Create a KFold object with 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=2023)

# Create a linear regression model
lr_model = LinearRegression()

# Perform k-Fold Cross-Validation and calculate the R-squared score for each fold
kfold_scores = cross_val_score(lr_model, predictors, target, cv=kf, scoring='r2')

# Calculate the average R-squared score across all k-Fold Cross-Validation iterations
kfold_avg_score = kfold_scores.mean()
print(f"k-Fold Average R-squared: {kfold_avg_score}")